# Dockstring

- tool wrapped around AutoDock to fully automate docking process pipeline, SMILES to docking score
- available at https://github.com/dockstring/dockstring , follow the install instructions, requires OpenBabel


In [1]:
from rdkit import Chem
from rdkit.Chem.SaltRemover import SaltRemover
from rdkit.Chem import Lipinski
from rdkit.Chem import rdMolDescriptors
from rdkit.Chem import Descriptors

import dockstring

import re
import csv
import pandas as pd
import numpy as np
import gzip

# Standardization

In [2]:
""" contribution from Hans de Winter """
def _InitialiseNeutralisationReactions():
    patts= (
        # Imidazoles
        ('[n+;H]','n'),
        # Amines
        ('[N+;!H0]','N'),
        # Carboxylic acids and alcohols
        ('[$([O-]);!$([O-][#7])]','O'),
        # Thiols
        ('[S-;X1]','S'),
        # Sulfonamides
        ('[$([N-;X2]S(=O)=O)]','N'),
        # Enamines
        ('[$([N-;X2][C,N]=C)]','N'),
        # Tetrazoles
        ('[n-]','[nH]'),
        # Sulfoxides
        ('[$([S-]=O)]','S'),
        # Amides
        ('[$([N-]C=O)]','N'),
        )
    return [(Chem.MolFromSmarts(x),Chem.MolFromSmiles(y,False)) for x,y in patts]

_reactions=None
def NeutraliseCharges(mol, reactions=None):
    global _reactions
    if reactions is None:
        if _reactions is None:
            _reactions=_InitialiseNeutralisationReactions()
        reactions=_reactions
    replaced = False
    for i,(reactant, product) in enumerate(reactions):
        while mol.HasSubstructMatch(reactant):
            replaced = True
            rms = Chem.ReplaceSubstructs(mol, reactant, product)
            mol = rms[0]
    return mol, replaced

In [3]:
_saltRemover = SaltRemover()
_inorganicPatt = Chem.MolFromSmarts("[!#6;!#7;!#8;!#16;!F;!Cl;!Br;!I]") # to remove compounds with unwanted atom types
_carbonPatt = Chem.MolFromSmarts("[#6]") # to remove compounds without carbon - inorganic
def standardize(mol):
    if mol.HasSubstructMatch(_carbonPatt):
        mol = _saltRemover(mol)
        if mol.GetNumAtoms()==0:
            return None
        else:
            mol, neutralized = NeutraliseCharges(mol)
            if mol.HasSubstructMatch(_inorganicPatt):
                return None
            else:
                # Sanitize mol, without sanitization some structures can't be drawn or fingerprinted
                Chem.SanitizeMol(mol) # add catch block?
                return mol
    else:
        return None

# Data loading

In [4]:
mols = []

In [5]:
# some pChEMBL values in our file are not valid floats
float_pattern = re.compile("[0-9]+(\.[0-9]*)?")
# function for Lipinski's descriptors
# load rock1, we want to remember source of data and Standard and pChEMBLE value (it's transfomed compound activity to "standard" value)
with open("../data/rock1/rock1_ic50_chembl.csv") as inp:
    rock1 = [row for row in csv.DictReader(inp, delimiter=";", quotechar='"')]
    for m in rock1:
        moldict = {"smiles": m["Smiles"]}
        mol = Chem.MolFromSmiles(moldict["smiles"])
        if mol:
            moldict["rdmol"] = mol
            stdmol = standardize(mol)
            moldict["source"] = "rock1_chembl"
            moldict["pChEMBL Value"] = float(m["pChEMBL Value"]) if float_pattern.match(m["pChEMBL Value"]) else 0
            moldict["std_rdmol"] = None
            moldict["Standard Value"] = float(m["Standard Value"]) if float_pattern.match(m["Standard Value"]) else np.NaN
        if stdmol:
            moldict["std_rdmol"] = stdmol
            moldict["std_smiles"] = Chem.MolToSmiles(stdmol)
            mols.append(moldict)

In [6]:
mols

[{'smiles': 'O=C(Nc1cccc(-c2ccnc3cc(-c4ccncc4)nn23)c1)c1cccc(C(F)(F)F)c1',
  'rdmol': <rdkit.Chem.rdchem.Mol at 0x7f16924d44e0>,
  'source': 'rock1_chembl',
  'pChEMBL Value': 0,
  'std_rdmol': <rdkit.Chem.rdchem.Mol at 0x7f1691f91530>,
  'Standard Value': 50000.0,
  'std_smiles': 'O=C(Nc1cccc(-c2ccnc3cc(-c4ccncc4)nn23)c1)c1cccc(C(F)(F)F)c1'},
 {'smiles': 'C[C@@H](Nc1c(Nc2ccncc2)c(=O)c1=O)c1ccccc1',
  'rdmol': <rdkit.Chem.rdchem.Mol at 0x7f1691f91b20>,
  'source': 'rock1_chembl',
  'pChEMBL Value': 7.22,
  'std_rdmol': <rdkit.Chem.rdchem.Mol at 0x7f1691f92120>,
  'Standard Value': 60.0,
  'std_smiles': 'C[C@@H](Nc1c(Nc2ccncc2)c(=O)c1=O)c1ccccc1'},
 {'smiles': 'C[C@@H](N)[C@H]1CC[C@H](C(=O)Nc2ccncc2)CC1',
  'rdmol': <rdkit.Chem.rdchem.Mol at 0x7f1691f92170>,
  'source': 'rock1_chembl',
  'pChEMBL Value': 6.82,
  'std_rdmol': <rdkit.Chem.rdchem.Mol at 0x7f1691f927b0>,
  'Standard Value': 150.0,
  'std_smiles': 'C[C@@H](N)[C@H]1CC[C@H](C(=O)Nc2ccncc2)CC1'},
 {'smiles': 'Cc1n[nH]c2ccc(-c3c

In [7]:
%%capture
# %%capture hides stderr (and maybe all output)
# same for drugbank
for m in Chem.SDMolSupplier("../data/drugbank.sdf"):
    if m:
        stdmol = standardize(m)
    else:
        continue    
    if stdmol:
        std_smiles = Chem.MolToSmiles(stdmol)
        mols.append({
         "rdmol": m,
         "std_rdmol": stdmol,
         "source": "drugbank",
         "std_smiles": std_smiles,
         "Smiles": Chem.MolToSmiles(m),
         "pChEMBL Value": 0,
         "Standard Value": np.NaN,
        })

[11:20:58] Explicit valence for atom # 7 N, 4, is greater than permitted
[11:20:58] ERROR: Could not sanitize molecule ending on line 20009
[11:20:58] ERROR: Explicit valence for atom # 7 N, 4, is greater than permitted
[11:20:59] Can't kekulize mol.  Unkekulized atoms: 1 2 3 5 6 7 8 9 10

[11:20:59] ERROR: Could not sanitize molecule ending on line 250947
[11:20:59] ERROR: Can't kekulize mol.  Unkekulized atoms: 1 2 3 5 6 7 8 9 10

[11:20:59] Explicit valence for atom # 17 O, 3, is greater than permitted
[11:20:59] ERROR: Could not sanitize molecule ending on line 258130
[11:20:59] ERROR: Explicit valence for atom # 17 O, 3, is greater than permitted
[11:20:59] Can't kekulize mol.  Unkekulized atoms: 57 58 59 60 61 62 63 64 65

[11:20:59] ERROR: Could not sanitize molecule ending on line 261581
[11:20:59] ERROR: Can't kekulize mol.  Unkekulized atoms: 57 58 59 60 61 62 63 64 65

[11:20:59] Explicit valence for atom # 47 Mg, 4, is greater than permitted
[11:20:59] ERROR: Could not sani

[11:20:59] ERROR: Explicit valence for atom # 2 O, 3, is greater than permitted
[11:21:00] Explicit valence for atom # 6 B, 4, is greater than permitted
[11:21:00] ERROR: Could not sanitize molecule ending on line 541262
[11:21:00] ERROR: Explicit valence for atom # 6 B, 4, is greater than permitted
[11:21:00] Can't kekulize mol.  Unkekulized atoms: 2 3 17 19 20

[11:21:00] ERROR: Could not sanitize molecule ending on line 557517
[11:21:00] ERROR: Can't kekulize mol.  Unkekulized atoms: 2 3 17 19 20

[11:21:00] Can't kekulize mol.  Unkekulized atoms: 0 1 2 4 5 6 7 8 9 10

[11:21:00] ERROR: Could not sanitize molecule ending on line 561344
[11:21:00] ERROR: Can't kekulize mol.  Unkekulized atoms: 0 1 2 4 5 6 7 8 9 10

[11:21:00] Can't kekulize mol.  Unkekulized atoms: 10 11 12 13 19

[11:21:00] ERROR: Could not sanitize molecule ending on line 562947
[11:21:00] ERROR: Can't kekulize mol.  Unkekulized atoms: 10 11 12 13 19

[11:21:00] Explicit valence for atom # 27 N, 4, is greater than 

In [8]:
%%capture
with gzip.open('../data/rock1/actives_final.sdf.gz', 'r') as sdf:
    suppl = Chem.ForwardSDMolSupplier(sdf)
    for m in suppl:
        print(m)
        if m:
            stdmol = standardize(m)
        else:
            continue    
        if stdmol:
            std_smiles = Chem.MolToSmiles(stdmol)
            mols.append({
             "rdmol": m,
             "std_rdmol": stdmol,
             "source": "rock1_active",
             "std_smiles": std_smiles,
             "Smiles": Chem.MolToSmiles(m),
             "pChEMBL Value": 0,
             "Standard Value": np.NaN,
            })

In [9]:
%%capture
with gzip.open('../data/rock1/decoys_final.sdf.gz', 'r') as sdf:
    suppl = Chem.ForwardSDMolSupplier(sdf)
    for m in suppl:
        print(m)
        if m:
            stdmol = standardize(m)
        else:
            continue    
        if stdmol:
            std_smiles = Chem.MolToSmiles(stdmol)
            mols.append({
             "rdmol": m,
             "std_rdmol": stdmol,
             "source": "rock1_decoy",
             "std_smiles": std_smiles,
             "Smiles": Chem.MolToSmiles(m),
             "pChEMBL Value": 0,
             "Standard Value": np.NaN,
            })

In [10]:
df = pd.DataFrame(mols)
df

,smiles,rdmol,source,pChEMBL Value,std_rdmol,Standard Value,std_smiles,Smiles
0,O=C(Nc1cccc(-c2ccnc3cc(-c4ccncc4)nn23)c1)c1ccc...,<rdkit.Chem.rdchem.Mol object at 0x7f16924d44e0>,rock1_chembl,0.00,<rdkit.Chem.rdchem.Mol object at 0x7f1691f91530>,50000.0,O=C(Nc1cccc(-c2ccnc3cc(-c4ccncc4)nn23)c1)c1ccc...,NaN
1,C[C@@H](Nc1c(Nc2ccncc2)c(=O)c1=O)c1ccccc1,<rdkit.Chem.rdchem.Mol object at 0x7f1691f91b20>,rock1_chembl,7.22,<rdkit.Chem.rdchem.Mol object at 0x7f1691f92120>,60.0,C[C@@H](Nc1c(Nc2ccncc2)c(=O)c1=O)c1ccccc1,NaN
2,C[C@@H](N)[C@H]1CC[C@H](C(=O)Nc2ccncc2)CC1,<rdkit.Chem.rdchem.Mol object at 0x7f1691f92170>,rock1_chembl,6.82,<rdkit.Chem.rdchem.Mol object at 0x7f1691f927b0>,150.0,C[C@@H](N)[C@H]1CC[C@H](C(=O)Nc2ccncc2)CC1,NaN
3,Cc1n[nH]c2ccc(-c3cc(OC[C@@H](N)Cc4ccccc4)cnc3-...,<rdkit.Chem.rdchem.Mol object at 0x7f1691f92800>,rock1_chembl,5.22,<rdkit.Chem.rdchem.Mol object at 0x7f1691f92df0>,6000.0,Cc1n[nH]c2ccc(-c3cc(OC[C@@H](N)Cc4ccccc4)cnc3-...,NaN
4,Cc1n[nH]c2ccc(-c3cc(OC[C@@H](N)Cc4ccccc4)cnc3-...,<rdkit.Chem.rdchem.Mol object at 0x7f1691f92e40>,rock1_chembl,0.00,<rdkit.Chem.rdchem.Mol object at 0x7f1691f97490>,10000.0,Cc1n[nH]c2ccc(-c3cc(OC[C@@H](N)Cc4ccccc4)cnc3-...,NaN
...,...,...,...,...,...,...,...,...
14143,NaN,<rdkit.Chem.rdchem.Mol object at 0x7f168bc19c10>,rock1_decoy,0.00,<rdkit.Chem.rdchem.Mol object at 0x7f168bc18990>,NaN,CCCCC(NCC(=O)Nc1ccc(F)c(F)c1F)C(=O)Nc1cc(C)on1,CCCCC([NH2+]CC(=O)Nc1ccc(F)c(F)c1F)C(=O)Nc1cc(...
14144,NaN,<rdkit.Chem.rdchem.Mol object at 0x7f168bc19ad0>,rock1_decoy,0.00,<rdkit.Chem.rdchem.Mol object at 0x7f168bc18750>,NaN,COc1cc(CNC(=O)NCC(C2CCCCC2)N(C)C)cc(OC)c1O,COc1cc(CNC(=O)NCC(C2CCCCC2)[NH+](C)C)cc(OC)c1O
14145,NaN,<rdkit.Chem.rdchem.Mol object at 0x7f168bc19f80>,rock1_decoy,0.00,<rdkit.Chem.rdchem.Mol object at 0x7f168bc18390>,NaN,COc1ccccc1C(CNC(=O)NNC(=O)CC1CCCCC1)N(C)C,COc1ccccc1C(CNC(=O)NNC(=O)CC1CCCCC1)[NH+](C)C
14146,NaN,<rdkit.Chem.rdchem.Mol object at 0x7f168bc19a80>,rock1_decoy,0.00,<rdkit.Chem.rdchem.Mol object at 0x7f168bc18090>,NaN,COc1ccccc1C(=O)NNC(=O)NCC(C(C)C)N1CCCCCC1,COc1ccccc1C(=O)NNC(=O)NCC(C(C)C)[NH+]1CCCCCC1


# Docking prep

In [11]:
def run_dockstring(target, ligand_smiles):
    """wrapper around Dockstring to return None if the process fails"""
    try:
        docking = target.dock(ligand_smiles)
        return docking
    except:
        return None, None

In [12]:
target = dockstring.load_target('ROCK1')
run_dockstring(target, 'CCO')

/home/uzivatel/miniconda3/envs/cheminfo/lib/python3.7/site-packages/dockstring/utils.py:265: UserWarning: Incorrect rdkit version 2020.09.1 (expected 2021.03.*). You can still run docking, but be advised that the output may not match the dockstring dataset.
  warnings.warn(message=f"Incorrect rdkit version {rdkit.__version__} "


(-2.4,
 {'ligand': <rdkit.Chem.rdchem.Mol at 0x7f16903ba2f0>,
  'affinities': [-2.4, -2.4, -2.3, -2.3, -2.2, -2.1, -2.0, -2.0, -1.9]})

# Docking

In [15]:
# stratified sampling, 10 from each category
df_sample = df.groupby('source', group_keys=False).apply(lambda x: x.sample(min(len(x), 10)))
df_sample

,smiles,rdmol,source,pChEMBL Value,std_rdmol,Standard Value,std_smiles,Smiles
6939,NaN,<rdkit.Chem.rdchem.Mol object at 0x7f1690209940>,drugbank,0.00,<rdkit.Chem.rdchem.Mol object at 0x7f16902093f0>,NaN,NC(=O)c1ccc(Nc2nc(N)c(N=O)c(OCC3CCCCC3)n2)cc1,NC(=O)c1ccc(Nc2nc(N)c(N=O)c(OCC3CCCCC3)n2)cc1
2993,NaN,<rdkit.Chem.rdchem.Mol object at 0x7f169050cb70>,drugbank,0.00,<rdkit.Chem.rdchem.Mol object at 0x7f169057d620>,NaN,COc1ccc(C2CN=C(O)C2)cc1OC1CCCC1,COc1ccc(C2CN=C(O)C2)cc1OC1CCCC1
7509,NaN,<rdkit.Chem.rdchem.Mol object at 0x7f16901deda0>,drugbank,0.00,<rdkit.Chem.rdchem.Mol object at 0x7f16901edc10>,NaN,N=C1CCCN1Cc1[nH]c(=O)[nH]c(=O)c1Cl,N=C1CCCN1Cc1[nH]c(=O)[nH]c(=O)c1Cl
5086,NaN,<rdkit.Chem.rdchem.Mol object at 0x7f16903a2080>,drugbank,0.00,<rdkit.Chem.rdchem.Mol object at 0x7f16903a2800>,NaN,Cn1cncc1[C@@](N)(c1ccc(Cl)cc1)c1ccc2c(c1)c(-c1...,Cn1cncc1[C@@](N)(c1ccc(Cl)cc1)c1ccc2c(c1)c(-c1...
5614,NaN,<rdkit.Chem.rdchem.Mol object at 0x7f169030cbc0>,drugbank,0.00,<rdkit.Chem.rdchem.Mol object at 0x7f169030c710>,NaN,O=C(O)Cn1ccc(=O)n(Cc2nc3cc(Cl)ccc3s2)c1=O,O=C(O)Cn1ccc(=O)n(Cc2nc3cc(Cl)ccc3s2)c1=O
3138,NaN,<rdkit.Chem.rdchem.Mol object at 0x7f169050ec10>,drugbank,0.00,<rdkit.Chem.rdchem.Mol object at 0x7f1690505f30>,NaN,C[C@@H](O)CO,C[C@@H](O)CO
5776,NaN,<rdkit.Chem.rdchem.Mol object at 0x7f1690331e90>,drugbank,0.00,<rdkit.Chem.rdchem.Mol object at 0x7f1690331580>,NaN,O=C(O)CCn1c2ccc(O)cc2c2c3c(c(-c4ccccc4)cc21)C(...,O=C(O)CCn1c2ccc(O)cc2c2c3c(c(-c4ccccc4)cc21)C(...
7538,NaN,<rdkit.Chem.rdchem.Mol object at 0x7f16901f03f0>,drugbank,0.00,<rdkit.Chem.rdchem.Mol object at 0x7f16901f0440>,NaN,O=C1CC[C@@H](C(=O)N2CSC[C@H]2C(=O)O)N1,O=C1CC[C@@H](C(=O)N2CSC[C@H]2C(=O)O)N1
2010,NaN,<rdkit.Chem.rdchem.Mol object at 0x7f16905c6080>,drugbank,0.00,<rdkit.Chem.rdchem.Mol object at 0x7f1691e3db20>,NaN,O=[N+]([O-])OCC(CO[N+](=O)[O-])O[N+](=O)[O-],O=[N+]([O-])OCC(CO[N+](=O)[O-])O[N+](=O)[O-]
4492,NaN,<rdkit.Chem.rdchem.Mol object at 0x7f169043ab70>,drugbank,0.00,<rdkit.Chem.rdchem.Mol object at 0x7f169043a6c0>,NaN,OC[C@@H]1O[C@@H](O)[C@H](O)[C@@H](O)[C@@H]1O[C...,OC[C@@H]1O[C@@H](O)[C@H](O)[C@@H](O)[C@@H]1O[C...


In [14]:
df_sample['rock1_dockstring_score'] = df_sample['std_smiles'].apply(lambda x: run_dockstring(target, x)[0])
df_sample

/home/uzivatel/miniconda3/envs/cheminfo/lib/python3.7/site-packages/dockstring/utils.py:265: UserWarning: Incorrect rdkit version 2020.09.1 (expected 2021.03.*). You can still run docking, but be advised that the output may not match the dockstring dataset.
  warnings.warn(message=f"Incorrect rdkit version {rdkit.__version__} "
/home/uzivatel/miniconda3/envs/cheminfo/lib/python3.7/site-packages/dockstring/utils.py:265: UserWarning: Incorrect rdkit version 2020.09.1 (expected 2021.03.*). You can still run docking, but be advised that the output may not match the dockstring dataset.
  warnings.warn(message=f"Incorrect rdkit version {rdkit.__version__} "
/home/uzivatel/miniconda3/envs/cheminfo/lib/python3.7/site-packages/dockstring/utils.py:265: UserWarning: Incorrect rdkit version 2020.09.1 (expected 2021.03.*). You can still run docking, but be advised that the output may not match the dockstring dataset.
  warnings.warn(message=f"Incorrect rdkit version {rdkit.__version__} "
/home/uziv

/home/uzivatel/miniconda3/envs/cheminfo/lib/python3.7/site-packages/dockstring/utils.py:265: UserWarning: Incorrect rdkit version 2020.09.1 (expected 2021.03.*). You can still run docking, but be advised that the output may not match the dockstring dataset.
  warnings.warn(message=f"Incorrect rdkit version {rdkit.__version__} "
/home/uzivatel/miniconda3/envs/cheminfo/lib/python3.7/site-packages/dockstring/utils.py:265: UserWarning: Incorrect rdkit version 2020.09.1 (expected 2021.03.*). You can still run docking, but be advised that the output may not match the dockstring dataset.
  warnings.warn(message=f"Incorrect rdkit version {rdkit.__version__} "
/home/uzivatel/miniconda3/envs/cheminfo/lib/python3.7/site-packages/dockstring/utils.py:265: UserWarning: Incorrect rdkit version 2020.09.1 (expected 2021.03.*). You can still run docking, but be advised that the output may not match the dockstring dataset.
  warnings.warn(message=f"Incorrect rdkit version {rdkit.__version__} "
/home/uziv

,smiles,rdmol,source,pChEMBL Value,std_rdmol,Standard Value,std_smiles,Smiles,rock1_dockstring_score
3478,NaN,<rdkit.Chem.rdchem.Mol object at 0x7f16904dbf80>,drugbank,0.00,<rdkit.Chem.rdchem.Mol object at 0x7f16904d9120>,NaN,O=C(O)C(=O)Nc1ccc(I)cc1C(=O)O,O=C(O)C(=O)Nc1ccc(I)cc1C(=O)O,-6.1
2043,NaN,<rdkit.Chem.rdchem.Mol object at 0x7f16905d0a80>,drugbank,0.00,<rdkit.Chem.rdchem.Mol object at 0x7f16905c8f30>,NaN,CCc1c2c(nc3ccc(OC(=O)N4CCC(N5CCCCC5)CC4)cc13)-...,CCc1c2c(nc3ccc(OC(=O)N4CCC(N5CCCCC5)CC4)cc13)-...,-9.6
3748,NaN,<rdkit.Chem.rdchem.Mol object at 0x7f16904a1300>,drugbank,0.00,<rdkit.Chem.rdchem.Mol object at 0x7f16904a17b0>,NaN,C[C@@H](O)C(=O)O,C[C@@H](O)C(=O)O,-3.8
6392,NaN,<rdkit.Chem.rdchem.Mol object at 0x7f1690299530>,drugbank,0.00,<rdkit.Chem.rdchem.Mol object at 0x7f1690294080>,NaN,CC#CCOc1ccc(S(=O)(=O)N2CCSC(C)(C)[C@@H]2C(=O)N...,CC#CCOc1ccc(S(=O)(=O)N2CCSC(C)(C)[C@@H]2C(=O)N...,-7.0
5338,NaN,<rdkit.Chem.rdchem.Mol object at 0x7f16903578a0>,drugbank,0.00,<rdkit.Chem.rdchem.Mol object at 0x7f1690357e40>,NaN,CCNC(=NCCCC[C@H](NC(=O)[C@H](CC(C)C)NC(=O)[C@@...,CCNC(=NCCCC[C@H](NC(=O)[C@H](CC(C)C)NC(=O)[C@@...,NaN
7135,NaN,<rdkit.Chem.rdchem.Mol object at 0x7f1690209170>,drugbank,0.00,<rdkit.Chem.rdchem.Mol object at 0x7f169021aad0>,NaN,Nc1ncc(/N=N/c2ccccc2)c(-c2ccc(Cl)cc2Cl)n1,Nc1ncc(/N=N/c2ccccc2)c(-c2ccc(Cl)cc2Cl)n1,-8.6
3879,NaN,<rdkit.Chem.rdchem.Mol object at 0x7f16904a5620>,drugbank,0.00,<rdkit.Chem.rdchem.Mol object at 0x7f16904a51c0>,NaN,C[C@H]1O[C@H](n2ccc(=O)[nH]c2=O)C[C@@H]1O,C[C@H]1O[C@H](n2ccc(=O)[nH]c2=O)C[C@@H]1O,-6.6
6787,NaN,<rdkit.Chem.rdchem.Mol object at 0x7f16902658a0>,drugbank,0.00,<rdkit.Chem.rdchem.Mol object at 0x7f1690265ee0>,NaN,COc1cc2ncnc(N3CC[C@@H](Oc4ccc5ccccc5c4)C3)c2cc1OC,COc1cc2ncnc(N3CC[C@@H](Oc4ccc5ccccc5c4)C3)c2cc1OC,-9.5
6055,NaN,<rdkit.Chem.rdchem.Mol object at 0x7f16902e4490>,drugbank,0.00,<rdkit.Chem.rdchem.Mol object at 0x7f16902e4b20>,NaN,CCc1nc(N)nc(N)c1-c1ccccc1,CCc1nc(N)nc(N)c1-c1ccccc1,-7.0
7390,NaN,<rdkit.Chem.rdchem.Mol object at 0x7f16901db670>,drugbank,0.00,<rdkit.Chem.rdchem.Mol object at 0x7f16901db7b0>,NaN,CCC(=O)N(c1ccccc1)[C@@]1(C(=O)OC)CCN(CCc2ccccc...,CCC(=O)N(c1ccccc1)[C@@]1(C(=O)OC)CCN(CCc2ccccc...,-8.2


In [16]:
df_sample2 = df.groupby('source', group_keys=False).apply(lambda x: x.sample(min(len(x), 30)))
df_sample2['rock1_dockstring_score'] = df_sample2['std_smiles'].apply(lambda x: run_dockstring(target, x)[0])
df_sample2

/home/uzivatel/miniconda3/envs/cheminfo/lib/python3.7/site-packages/dockstring/utils.py:265: UserWarning: Incorrect rdkit version 2020.09.1 (expected 2021.03.*). You can still run docking, but be advised that the output may not match the dockstring dataset.
  warnings.warn(message=f"Incorrect rdkit version {rdkit.__version__} "
/home/uzivatel/miniconda3/envs/cheminfo/lib/python3.7/site-packages/dockstring/utils.py:265: UserWarning: Incorrect rdkit version 2020.09.1 (expected 2021.03.*). You can still run docking, but be advised that the output may not match the dockstring dataset.
  warnings.warn(message=f"Incorrect rdkit version {rdkit.__version__} "
/home/uzivatel/miniconda3/envs/cheminfo/lib/python3.7/site-packages/dockstring/utils.py:265: UserWarning: Incorrect rdkit version 2020.09.1 (expected 2021.03.*). You can still run docking, but be advised that the output may not match the dockstring dataset.
  warnings.warn(message=f"Incorrect rdkit version {rdkit.__version__} "
/home/uziv

/home/uzivatel/miniconda3/envs/cheminfo/lib/python3.7/site-packages/dockstring/utils.py:265: UserWarning: Incorrect rdkit version 2020.09.1 (expected 2021.03.*). You can still run docking, but be advised that the output may not match the dockstring dataset.
  warnings.warn(message=f"Incorrect rdkit version {rdkit.__version__} "
/home/uzivatel/miniconda3/envs/cheminfo/lib/python3.7/site-packages/dockstring/utils.py:265: UserWarning: Incorrect rdkit version 2020.09.1 (expected 2021.03.*). You can still run docking, but be advised that the output may not match the dockstring dataset.
  warnings.warn(message=f"Incorrect rdkit version {rdkit.__version__} "
/home/uzivatel/miniconda3/envs/cheminfo/lib/python3.7/site-packages/dockstring/utils.py:265: UserWarning: Incorrect rdkit version 2020.09.1 (expected 2021.03.*). You can still run docking, but be advised that the output may not match the dockstring dataset.
  warnings.warn(message=f"Incorrect rdkit version {rdkit.__version__} "
/home/uziv

/home/uzivatel/miniconda3/envs/cheminfo/lib/python3.7/site-packages/dockstring/utils.py:265: UserWarning: Incorrect rdkit version 2020.09.1 (expected 2021.03.*). You can still run docking, but be advised that the output may not match the dockstring dataset.
  warnings.warn(message=f"Incorrect rdkit version {rdkit.__version__} "
/home/uzivatel/miniconda3/envs/cheminfo/lib/python3.7/site-packages/dockstring/utils.py:265: UserWarning: Incorrect rdkit version 2020.09.1 (expected 2021.03.*). You can still run docking, but be advised that the output may not match the dockstring dataset.
  warnings.warn(message=f"Incorrect rdkit version {rdkit.__version__} "
/home/uzivatel/miniconda3/envs/cheminfo/lib/python3.7/site-packages/dockstring/utils.py:265: UserWarning: Incorrect rdkit version 2020.09.1 (expected 2021.03.*). You can still run docking, but be advised that the output may not match the dockstring dataset.
  warnings.warn(message=f"Incorrect rdkit version {rdkit.__version__} "
/home/uziv

/home/uzivatel/miniconda3/envs/cheminfo/lib/python3.7/site-packages/dockstring/utils.py:265: UserWarning: Incorrect rdkit version 2020.09.1 (expected 2021.03.*). You can still run docking, but be advised that the output may not match the dockstring dataset.
  warnings.warn(message=f"Incorrect rdkit version {rdkit.__version__} "
/home/uzivatel/miniconda3/envs/cheminfo/lib/python3.7/site-packages/dockstring/utils.py:265: UserWarning: Incorrect rdkit version 2020.09.1 (expected 2021.03.*). You can still run docking, but be advised that the output may not match the dockstring dataset.
  warnings.warn(message=f"Incorrect rdkit version {rdkit.__version__} "
/home/uzivatel/miniconda3/envs/cheminfo/lib/python3.7/site-packages/dockstring/utils.py:265: UserWarning: Incorrect rdkit version 2020.09.1 (expected 2021.03.*). You can still run docking, but be advised that the output may not match the dockstring dataset.
  warnings.warn(message=f"Incorrect rdkit version {rdkit.__version__} "
/home/uziv

/home/uzivatel/miniconda3/envs/cheminfo/lib/python3.7/site-packages/dockstring/utils.py:265: UserWarning: Incorrect rdkit version 2020.09.1 (expected 2021.03.*). You can still run docking, but be advised that the output may not match the dockstring dataset.
  warnings.warn(message=f"Incorrect rdkit version {rdkit.__version__} "
RDKit ERROR: [11:21:02] ERROR: Explicit valence for atom # 0 C greater than permitted
RDKit ERROR: [11:41:51] Explicit valence for atom # 18 N, 4, is greater than permitted
[11:21:02] ERROR: Explicit valence for atom # 0 C greater than permitted
[11:41:51] Explicit valence for atom # 18 N, 4, is greater than permitted
/home/uzivatel/miniconda3/envs/cheminfo/lib/python3.7/site-packages/dockstring/utils.py:265: UserWarning: Incorrect rdkit version 2020.09.1 (expected 2021.03.*). You can still run docking, but be advised that the output may not match the dockstring dataset.
  warnings.warn(message=f"Incorrect rdkit version {rdkit.__version__} "
/home/uzivatel/minic

,smiles,rdmol,source,pChEMBL Value,std_rdmol,Standard Value,std_smiles,Smiles,rock1_dockstring_score
6866,NaN,<rdkit.Chem.rdchem.Mol object at 0x7f169024eb70>,drugbank,0.0,<rdkit.Chem.rdchem.Mol object at 0x7f1690253490>,NaN,N=C(N)c1cccc(CN2CCN(S(=O)(=O)c3cc4ccc(Cl)cc4s3...,N=C(N)c1cccc(CN2CCN(S(=O)(=O)c3cc4ccc(Cl)cc4s3...,-9.4
4190,NaN,<rdkit.Chem.rdchem.Mol object at 0x7f169044dad0>,drugbank,0.0,<rdkit.Chem.rdchem.Mol object at 0x7f1690446a80>,NaN,COc1cc[nH]c1/C=C1\C(=O)Nc2ccc(F)c(C#C[C@@H](O)...,COc1cc[nH]c1/C=C1\C(=O)Nc2ccc(F)c(C#C[C@@H](O)...,-8.3
4838,NaN,<rdkit.Chem.rdchem.Mol object at 0x7f16903e5f30>,drugbank,0.0,<rdkit.Chem.rdchem.Mol object at 0x7f16903e56c0>,NaN,CCCNCCCNCCCNCC,CCCNCCCNCCCNCC,-4.4
3559,NaN,<rdkit.Chem.rdchem.Mol object at 0x7f1690485f80>,drugbank,0.0,<rdkit.Chem.rdchem.Mol object at 0x7f16904e98f0>,NaN,C[C@@H]1CC[C@H](O)[C@H]1NC(=O)[C@@H](Cc1ccccc1...,C[C@@H]1CC[C@H](O)[C@H]1NC(=O)[C@@H](Cc1ccccc1...,-10.1
5301,NaN,<rdkit.Chem.rdchem.Mol object at 0x7f169035a2b0>,drugbank,0.0,<rdkit.Chem.rdchem.Mol object at 0x7f1690355270>,NaN,O=C(O)O,O=C([O-])[O-].[Ca+2],-3.4
...,...,...,...,...,...,...,...,...,...
7998,NaN,<rdkit.Chem.rdchem.Mol object at 0x7f16901db940>,rock1_decoy,0.0,<rdkit.Chem.rdchem.Mol object at 0x7f169015a390>,NaN,C=CCOc1ccc(/C=C(\C#N)C2N=c3cc(C)c(C)cc3=N2)cc1OC,C=CCOc1ccc(/C=C(\C#N)C2[NH+]=c3cc(C)c(C)cc3=[N...,-8.3
12259,NaN,<rdkit.Chem.rdchem.Mol object at 0x7f168be0c760>,rock1_decoy,0.0,<rdkit.Chem.rdchem.Mol object at 0x7f168bdfe030>,NaN,CCCCc1ccc(N2C(=O)C3C(CCC(=O)O)NC4(C(=O)Nc5cccc...,CCCCc1ccc(N2C(=O)C3C(CCC(=O)[O-])NC4(C(=O)Nc5c...,-9.4
13130,NaN,<rdkit.Chem.rdchem.Mol object at 0x7f168bd2eb70>,rock1_decoy,0.0,<rdkit.Chem.rdchem.Mol object at 0x7f168bd1f6f0>,NaN,CCc1nc(C2C(c3ccccc3)NCN2CCCc2nnn[nH]2)c(C)[nH]1,CCc1[nH]c(C)c(C2C(c3ccccc3)NC[NH+]2CCCc2nnn[n-...,NaN
10002,NaN,<rdkit.Chem.rdchem.Mol object at 0x7f168bfe6850>,rock1_decoy,0.0,<rdkit.Chem.rdchem.Mol object at 0x7f168bff76f0>,NaN,CCN1CCN(c2ccc(NC(=O)NCC(=O)NC(C)(C)C)c(C)c2)CC1,CC[NH+]1CCN(c2ccc(NC(=O)NCC(=O)NC(C)(C)C)c(C)c...,-7.4
